## Optimization 1 Individual Assignment 
By: Stephanie Palanca

a) provide mathematical formulation for the production/stock policy problem with the goal of minimizing the total cost

Min (440 * sum(pi)) + (700 * sum(qi)) + (5* sum(lo))
- pi = max production for month i = 1, 2, 3, 4
- qi = max extra production for month i = 1, 2, 3, 4
- lo = what is left over for each month for month i = 1, 2, 3, 4

B and C)

(ii) code the problem using Python and Gurobi and include the code in your answer, and

(iii) solve the problem that you have coded and report (a) the optimal monthly production amounts and (b) the total cost.

In [8]:
#B and C
#(ii) code the problem using Python and Gurobi and include the code in your answer, and
#(iii) solve the problem that you have coded and report (a) the optimal monthly production
#amounts and (b) the total cost.

import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum
import pandas as pd
import numpy as np

#creating arrays for data given in the table of the question (I will only be using the max_production, max_extra production, and demand arrays but the rest were added because it helped me visualize the problem)
month = np.array([1, 2, 3, 4])
max_production = np.array([140, 150, 140, 160])
max_production_cost = np.array([440, 440, 440, 440])
max_extra_production = np.array([50, 75, 70, 80])
demand = np.array([120, 160, 300, 200])
max_extra_production_cost = np.array([260, 260, 260, 260])

m = gp.Model('production')

###### ====== Decision Variables ======
p1 = m.addVar(name='Regular Production for Month 1')
p2 = m.addVar(name='Regular Production for Month 2')
p3 = m.addVar(name='Regular Production for Month 3')
p4 = m.addVar(name='Regular Production for Month 4')

q1 = m.addVar(name= 'Extra Production for Month 1')
q2 = m.addVar(name= 'Extra Production for Month 2')
q3 = m.addVar(name= 'Extra Production for Month 3')
q4 = m.addVar(name= 'Extra Production for Month 4')

lo1 = m.addVar(name='Left over from Month 1')
lo2 = m.addVar(name='Left over from Month 2')
lo3 = m.addVar(name='Left over from Month 3')
lo4 = m.addVar(name='Left over from Month 4')

###### ====== Objective Variable ======
obj = (440 * (p1+p2+p3+p4)) + (700 * (q1+q2+q3+q4)) + (5*(lo1 + lo2 + lo3 + lo4))
m.setObjective(obj, GRB.MINIMIZE)

#Defining excess (leftover) production for each month 
extra1 = p1 + q1 - demand[0]
extra2 = extra1 + (p2 + q2 - demand[1])
extra3 = extra2 + (p3 + q3 - demand[2])
extra4 = extra3 + (p4 + q4 - demand[3])

#Defining 10% of the sum of month 1, 2 and 3 max production
cap_con = 0.1*(p1 + p2 + p3)

###### ====== Constraints ======
##Warehouse Capacity Constraint - Maximum regular production for each month has to be at least 10% of the total production of the first 3 months
con1 = m.addConstr(p1 >= cap_con)
con2 = m.addConstr(p2 >= cap_con)
con3 = m.addConstr(p3 >= cap_con)
con4 = m.addConstr(p4 >= cap_con)

##Max Production constraints - Maximum regular production for each month has to be less than or equal to the corresponding numbers in the max_production array
#Max production for month 1 has to be less than or equal to 140
con5 = m.addConstr(p1 <= max_production[0])
#Max production for month 2 has to be less than or equal to 150
con6 = m.addConstr(p2 <= max_production[1])
#Max production for month 2 has to be less than or equal to 140
con7 = m.addConstr(p3 <= max_production[2])
#Max production for month 4 has to be less than or equal to 160
con8 = m.addConstr(p4 <= max_production[3])

##Max Extra Production Constraint
#Max extra production for month 1 must be less than or equal to 50
con9 = m.addConstr(q1 <= max_extra_production[0])
#Max extra production for month 2 must be less than or equal to 75
con10 = m.addConstr(q2 <= max_extra_production[1])
#Max extra production for month 2 must be less than or equal to 70
con11 = m.addConstr(q3 <= max_extra_production[2])
#Max extra production for month 2 must be less than or equal to 80
con12 = m.addConstr(q4 <= max_extra_production[3])

##"Left Over" Constraints
#For month 1-3, maximum storage capacity at the end of the month is 100 tons of rice
con13 = m.addConstr(lo1 <= 100)
con14 = m.addConstr(lo2 <= 100)
con15 = m.addConstr(lo3 <= 100)
#Warehouse must be empty at the end of month 4
con16 = m.addConstr(lo4 == 0)

##Decision variables for left over has to be equal to the excess equation specified above
con17 = m.addConstr(lo1 == extra1)
con18 = m.addConstr(lo2 == extra2)
con19 = m.addConstr(lo3 == extra3)
con20 = m.addConstr(lo4 == extra4)

##non negativity constraints for max_production and max_extra_production
con21 = m.addConstr(p1 >= 0)
con22 = m.addConstr(p2 >= 0)
con23 = m.addConstr(p3 >= 0)
con24 = m.addConstr(p4 >= 0)

con25 = m.addConstr(q1 >= 0)
con26 = m.addConstr(q2 >= 0)
con27 = m.addConstr(q3 >= 0)
con28 = m.addConstr(q4 >= 0)

###### ====== Running the model / reporting monthly optimal production & total cost ======
m.optimize()

print('\nTotal Cost: $ %g' % m.objVal)

# Print optimal values for the decision variables
print('\nMonthly Optimal Production:')
for v in m.getVars():
    print('%s = %g' % (v.varName, v.x))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: VirtualApple @ 2.50GHz processor
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 12 columns and 57 nonzeros
Model fingerprint: 0xefd62851
Coefficient statistics:
  Matrix range     [1e-01, 1e+00]
  Objective range  [5e+00, 7e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 8e+02]
Presolve removed 25 rows and 6 columns
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.5703210e+05   1.066950e+01   0.000000e+00      0s
       3    3.9317500e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.931750000e+05

Total Cost: $ 393175

Monthly Optimal Production:
Regular Production for Month 1 = 140
Regular Production for Month 2 = 150
Regular Production for Month 3 = 140
Regular Produ